# Generate Pre-Trained Models

This nootebook is responsible for training and saving in a .pkl file the pre-trained models for classification

### Import Statements

In [ ]:
import sys
import os
import pandas as pd

p = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(p) 

from DecisionTree.Bootstrap_Aggregating import Bagging
from DecisionTree.ID3Tree import ID3Tree
from DecisionTree.Ruleset import Ruleset

### Load Data

This loaded data is the data generated from generate_MCTS_dataset.ipynb file

In [ ]:
data_file = os.path.join(p, 'results', 'monte_carlo_data.csv')
data = pd.read_csv(data_file, delimiter=';')
data = data.drop(columns=['turn'])
data['played'] = data['played'].astype(int)
data

,cel1,cel2,cel3,cel4,cel5,cel6,cel7,cel8,cel9,cel10,...,cel35,cel36,cel37,cel38,cel39,cel40,cel41,cel42,pieces,played
0,0,0,0,0,0,0,0,0,0,0,...,-1,0,-1,-1,1,1,-1,1,12,5
1,0,0,0,0,0,0,0,0,0,0,...,1,0,-1,1,1,-1,-1,-1,13,5
2,0,0,0,0,0,0,0,0,0,0,...,0,0,-1,-1,1,-1,1,1,7,5
3,0,0,0,0,0,-1,0,0,0,0,...,-1,-1,1,0,1,-1,1,-1,17,3
4,0,0,0,0,0,0,0,0,0,0,...,-1,0,0,0,0,0,-1,1,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30012,0,0,0,0,0,0,0,0,0,0,...,0,-1,1,-1,-1,0,1,1,10,3
30013,0,0,0,0,0,0,0,0,0,0,...,0,0,-1,-1,1,0,1,0,9,3
30014,0,0,0,0,0,0,0,0,0,0,...,0,1,1,-1,-1,1,-1,1,17,1
30015,0,0,0,0,0,0,1,0,0,0,...,-1,1,-1,-1,1,-1,-1,1,21,2


In [ ]:
X, y = data.iloc[:, :-1], data.iloc[:, -1]
X = X.to_numpy().tolist()
y = y.to_numpy().tolist()
print(y)

[5, 5, 5, 3, 6, 4, 6, 3, 5, 2, 3, 2, 5, 4, 2, 4, 0, 3, 5, 4, 4, 4, 3, 3, 4, 4, 3, 5, 2, 4, 2, 5, 3, 4, 6, 1, 4, 4, 1, 2, 2, 1, 2, 1, 1, 5, 5, 3, 2, 6, 3, 1, 3, 2, 4, 3, 2, 4, 3, 6, 4, 2, 2, 6, 0, 1, 3, 2, 3, 3, 5, 4, 4, 1, 4, 1, 2, 0, 1, 5, 3, 1, 0, 1, 5, 3, 1, 2, 4, 0, 4, 1, 1, 4, 1, 5, 3, 6, 2, 1, 2, 4, 0, 4, 3, 3, 6, 3, 2, 3, 1, 2, 1, 3, 3, 4, 0, 6, 5, 3, 2, 3, 0, 0, 2, 4, 5, 5, 4, 3, 5, 3, 4, 2, 4, 4, 6, 1, 1, 5, 5, 4, 1, 3, 5, 6, 3, 4, 2, 3, 4, 4, 2, 4, 6, 4, 3, 2, 3, 3, 1, 1, 2, 3, 3, 2, 4, 3, 4, 4, 3, 5, 3, 4, 1, 3, 4, 0, 5, 1, 4, 2, 3, 2, 4, 2, 5, 3, 0, 0, 4, 5, 3, 4, 4, 5, 5, 2, 3, 3, 4, 5, 4, 1, 3, 5, 0, 4, 2, 6, 2, 2, 5, 1, 6, 4, 3, 5, 4, 3, 2, 4, 3, 1, 0, 3, 1, 3, 0, 4, 1, 3, 4, 2, 1, 6, 2, 2, 3, 6, 2, 4, 3, 3, 6, 3, 5, 5, 3, 4, 6, 2, 3, 4, 2, 3, 2, 3, 3, 6, 2, 4, 2, 2, 1, 3, 3, 4, 2, 5, 6, 3, 3, 3, 4, 1, 2, 5, 3, 5, 6, 4, 4, 5, 1, 2, 2, 0, 4, 4, 3, 2, 0, 3, 6, 4, 3, 3, 4, 1, 6, 5, 2, 4, 4, 2, 1, 3, 3, 0, 2, 3, 4, 6, 3, 3, 3, 3, 2, 2, 3, 3, 4, 3, 2, 2, 3, 5, 2, 2, 4, 3, 2, 

### Create init variables

In the next code cell, we will crete all necessery variables for the initialization of the different ID3 Tree classes

In [4]:
def attribute_vartypes(attribute):
    """
    Function to determine the attribute types of the dataset.
    :param X: The dataset
    :return: A list of attribute types
    """
    return 'continuous' if attribute == 'pieces' else 'categorical'

In [ ]:
data = [x + [label] for x, label in zip(X, y)]
feature_names = data.columns[:-1].tolist()
type_map = {attribute : attribute_vartypes(attribute) for attribute in feature_names}
print(data[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, -1, 1, 0, -1, 1, -1, 0, -1, -1, 1, 1, -1, 1, 12, 5]


### Set save folder

In [6]:
save_folder = os.path.join(p, 'models')

### ID3 Decision Tree Training & Saving 

In [ ]:
id3_tree = ID3Tree(feature_names, data, default=0, type_map=type_map)
id3_tree.train()
id3_tree.save_model(os.path.join(save_folder, 'id3_tree.pkl'))
print("ID3 Tree trained and saved successfully.")

'id3_tree = ID3Tree(feature_names, data, default=0, type_map=type_map)\nid3_tree.train()\nid3_tree.save_model(os.path.join(save_folder, \'id3_tree.pkl\'))\nprint("ID3 Tree trained and saved successfully.")'

### Tree with Prunning Training & Saving

In [ ]:
ruleset = Ruleset(feature_names, data, default=0, type_map=type_map)
ruleset.train()
ruleset.save_model(os.path.join(save_folder, 'ruleset.pkl'))
print("Ruleset trained and saved successfully.")

'ruleset = Ruleset(feature_names, data, default=0, type_map=type_map)\nruleset.train()\nruleset.save_model(os.path.join(save_folder, \'ruleset.pkl\'))\nprint("Ruleset trained and saved successfully.")'

### Bagging Training & Saving

In [ ]:
bagging = Bagging(feature_names, data, default=0, type_map=type_map)
bagging.train()
bagging.save_model(os.path.join(save_folder, 'bagging_model.pkl'))
print("Bagging model trained and saved successfully.")

Training classifier #1
Training classifier #2
Training classifier #3
Training classifier #4
Training classifier #5
Training classifier #6
Training classifier #7
Training classifier #8
Training classifier #9
Training classifier #10
Model saved to bagging_model.pkl
Bagging model trained and saved successfully.
